# World Bank data mining
We collect data from the WB using the Python package *wbgapi*, which makes accessing the API very simple

The variables were chosing using *domain knowledge* from previous work, thus were handpicked

Alternative approaches can include:
- Scraping all variables, ranking then by correlation (Pearson, Spearman, whatever you choose), and choosing the *n* highest
- Using the same variables as other author(s)
- Scraping all variables, and doing dimensionality reduction later (PCA, etcetera)
- Combination of all of the above

In [1]:
#!pip install wbgapi

In [4]:
import pandas as pd
import numpy as np

import wbgapi as wb
import requests

In [5]:
# for searching all indicators within a database
# wb.series.info(db=2)
x = 15
x

In [5]:
# for searching all indicators within a database
# wb.series.info(db=2)

In [6]:
# For searching specific indicators
wb.series.info(q='GDP')

id,value
EG.GDP.PUSE.KO.PP,GDP per unit of energy use (PPP $ per kg of oil equivalent)
EG.GDP.PUSE.KO.PP.KD,GDP per unit of energy use (constant 2017 PPP $ per kg of oil equivalent)
EG.USE.COMM.GD.PP.KD,"Energy use (kg of oil equivalent) per $1,000 GDP (constant 2017 PPP)"
NY.GDP.DEFL.KD.ZG,"Inflation, GDP deflator (annual %)"
NY.GDP.DEFL.KD.ZG.AD,"Inflation, GDP deflator: linked series (annual %)"
NY.GDP.DEFL.ZS,GDP deflator (base year varies by country)
NY.GDP.DEFL.ZS.AD,GDP deflator: linked series (base year varies by country)
NY.GDP.DISC.CN,Discrepancy in expenditure estimate of GDP (current LCU)
NY.GDP.DISC.KN,Discrepancy in expenditure estimate of GDP (constant LCU)
NY.GDP.MKTP.CD,GDP (current US$)


In [7]:
# collecting all keys and placing them into a nice dataframe - these are the labels of
# variables to be fed into the ML model, so it helps to be careful in this section:

vars_dict = {'code': ['NY.GDP.MKTP.KD.ZG', 'NY.GDP.MKTP.KD','NY.GDP.PCAP.KD',
                      'NE.CON.GOVT.ZS', 'NE.CON.PRVT.ZS', 'GC.NFN.TOTL.GD.ZS','NY.GDP.DEFL.KD.ZG', 'EG.FEC.RNEW.ZS',
                      'SL.UEM.TOTL.NE.ZS','SL.EMP.TOTL.SP.NE.ZS','HD.HCI.OVRL','SL.AGR.EMPL.ZS','SL.EMP.VULN.ZS',
                      # environment:
                      'EN.POP.SLUM.UR.ZS','ER.H2O.INTR.PC',
                      'NE.IMP.GNFS.ZS','NE.EXP.GNFS.ZS','BX.KLT.DINV.CD.WD','GC.DOD.TOTL.GD.ZS','FR.INR.RINR',
                      'SI.POV.DDAY','SI.POV.LMIC','SI.POV.UMIC','SI.POV.MDIM','SI.POV.MDIM.XQ',
                      'SI.POV.GINI','BN.CAB.XOKA.GD.ZS','SP.POP.TOTL','EN.POP.DNST','SP.POP.GROW',
                      'SP.DYN.LE00.IN','SP.DYN.CDRT.IN', 'SH.DYN.MORT','SH.DTH.COMM.ZS',
                      # agric:
                      'EG.ELC.RNEW.ZS','AG.LND.AGRI.ZS','NV.AGR.TOTL.ZS','AG.LND.ARBL.ZS','SP.RUR.TOTL.ZS',
                      # climate change:
                      'EG.ELC.ACCS.ZS','EG.USE.ELEC.KH.PC',
                      # energy & mining:
                      'TX.VAL.FUEL.ZS.UN','TX.VAL.MMTL.ZS.UN',
                      'EN.ATM.CO2E.PC','EN.CLC.GHGR.MT.CE','NV.IND.TOTL.ZS','NV.SRV.EMPL.KD',
                      'SE.ENR.PRSC.FM.ZS','SE.ADT.LITR.ZS','SE.TER.CUAT.BA.ZS','VC.IHR.PSRC.P5','SP.POP.SCIE.RD.P6'],
            'label': ['gdp_real_gwt', 'gdp_real_us_fixed','gdp_per_capita',
                      'gdp_pp_govt', 'gdp_pp_private', 'investment', 'inflation_pp', 'renew_energy',
                      'unemployment','employment','HCI','agric_employment','vulnerable_employment',
                      # environment:
                      'pop_slums','renew_freshwater',
                      'imports','exports','foreign_inv','govt_debt','real_interest_rate',
                      'poverty_1.90','poverty_3.20','poverty_5.50','poverty_multidim','poverty_mult_index',
                      'gini_index','cab','population','pop_density','pop_growth',
                      'life_expectancy','death_rate', 'child_mortality','cause_of_death',
                      # agric:
                      'renewable_energy_output','agric_land','agff_gdp','arable_land','rural_pop',
                      # climate change:
                      'electricity_access','power_consumption',
                      # energy & mining:
                      'fuel_exports','metal_exports',
                      'co2_emissions','ghg_emissions','industry_gdp','service_value_added',
                      'school_enroll','literacy','bachelor','homicide','research']}

vars_df = pd.DataFrame(vars_dict)

In [8]:
len(vars_df)

52

In [ ]:
# obtaining definition/description of each variable
vars_df = vars_df.assign(definition='')
for i in range(0,len(vars_df)):
    vars_df.iloc[i,2] = wb.series.get(id=vars_df.iloc[i,0])['value']

In [14]:
# function for obtaining a clean dataframe with columns "economy", "Year", and economic indicators:
def vert_df(gdp_df, name):
    gdp_df.columns = gdp_df.columns.str.replace('YR','')
    gdp_df = gdp_df.reset_index()
    gdp_df.drop('Country', inplace=True, axis=1) # dropping "Country" - we are using the ISO3 code instead
    year_drop = list(range(1960, 1990)) # dropping all years before 1990
    year_drop = [*map(str,year_drop)]
    gdp_df.drop(year_drop, inplace=True, axis=1)
    gdp_df = gdp_df.melt(id_vars = ['economy'], var_name = 'Year', value_name = name)
    gdp_df.Year = pd.to_numeric(gdp_df.Year)
    return gdp_df

In [ ]:
vars_df

In [15]:
# loop for creating dataframe to export:
gdp_df = vert_df(wb.data.DataFrame(vars_df.iloc[0,0], labels=True), vars_df.iloc[0,1])
for i in range(1, len(vars_df)):
    wbcode, wblabel = vars_df.iloc[i,0], vars_df.iloc[i,1]
    new_data = vert_df(wb.data.DataFrame(wbcode, labels=True), wblabel)
    gdp_df = pd.merge(gdp_df, new_data, how='left', on=['economy', 'Year'])

In [ ]:
# replacing label:
gdp_df.rename(columns = {'economy':'ISO3'}, inplace = True)

In [ ]:
gdp_df

In [18]:
gdp_df.to_csv('data/wb_data.csv', index=False)